<a href="https://colab.research.google.com/github/mohmiim/MLIntroduction/blob/master/session-2/Session_2_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf

# ugly
if ('rc' in tf.__version__) == True :
  !pip uninstall tensorflow
  !pip install tensorflow-gpu

print(tf.__version__)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD




2.1.0


In [0]:
# https://github.com/mohmiim/MLIntroduction/raw/master/data/data.zip
# https://github.com/mohmiim/MLIntroduction/raw/master/data/smallData.zip





In [3]:
import os

!wget --no-check-certificate \
    https://github.com/mohmiim/MLIntroduction/raw/master/data/smallData.zip \
    -O /tmp/smallData.zip

--2020-03-27 19:30:19--  https://github.com/mohmiim/MLIntroduction/raw/master/data/smallData.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mohmiim/MLIntroduction/master/data/smallData.zip [following]
--2020-03-27 19:30:20--  https://raw.githubusercontent.com/mohmiim/MLIntroduction/master/data/smallData.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7426985 (7.1M) [application/zip]
Saving to: ‘/tmp/smallData.zip’

/tmp/smallData.zip  100%[===================>]   7.08M  28.7MB/s    in 0.2s    

2020-03-27 19:30:20 (28.7 MB/s) - ‘/tmp/smallData.zip’ saved [7426985/7426985]



In [0]:
import zipfile
def unzip(file,target):
  zip_ref = zipfile.ZipFile(file, 'r')
  zip_ref.extractall(target)
  zip_ref.close()

unzip("/tmp/smallData.zip","/tmp")

In [5]:
if os.path.exists('/tmp/smallData'):
    print(os.listdir('/tmp/smallData'))


['test', 'train', '.gitignore']


In [0]:
#default sizes
Image_Width = 100
Image_Height = 100
Image_Depth = 3
targetSize = (Image_Width,Image_Height)
targetSize_withdepth = (Image_Width,Image_Height,Image_Depth)
CLASSES_COUNT = 2;
epochs = 50

In [7]:
#define the sub folders for both training and test
training = os.path.join("/tmp/smallData",'train')

#now the easiest way to load data is to use the ImageDataGenerator
train_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_data_generator.flow_from_directory(training,
                                                           batch_size=20,
                                                           target_size=targetSize,
                                                           #seed=12
                                                           )

Found 95 images belonging to 2 classes.


In [8]:
model = Sequential()
model.add(Flatten(input_shape=targetSize_withdepth))
model.add(Dense(1024,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(CLASSES_COUNT,activation='softmax'))
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 30000)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              30721024  
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 31,246,850
Trainable params: 31,246,850
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit_generator(generator=train_generator,epochs=epochs)


Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 5 steps
Epoch 1/50


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


5/5 [==============================] - 2s 401ms/step - loss: 4.2647 - accuracy: 0.5158
Epoch 2/50
5/5 [==============================] - 1s 140ms/step - loss: 0.5246 - accuracy: 0.7053
Epoch 3/50
5/5 [==============================] - 1s 134ms/step - loss: 0.2439 - accuracy: 0.9263
Epoch 4/50
5/5 [==============================] - 1s 134ms/step - loss: 0.5995 - accuracy: 0.6947
Epoch 5/50
5/5 [==============================] - 1s 138ms/step - loss: 0.1604 - accuracy: 0.9368
Epoch 6/50
5/5 [==============================] - 1s 132ms/step - loss: 0.3293 - accuracy: 0.8316
Epoch 7/50
5/5 [==============================] - 1s 135ms/step - loss: 0.0972 - accuracy: 0.9684
Epoch 8/50
5/5 [==============================] - 1s 132ms/step - loss: 0.0599 - accuracy: 0.9895
Epoch 9/50
5/5 [==============================] - 1s 136ms/step - loss: 0.0395 - accuracy: 1.0000
Epoch 10/50
5/5 [==============================] - 1s 134ms/step - loss: 0.0246 - accuracy: 1.0000
Epoch 11/50
5/5 [=============

In [0]:
from sklearn.metrics import confusion_matrix, classification_report
def test(generator, model):
  predictions = model.predict_generator(generator)
  row_index = predictions.argmax(axis=1)
  target_names = generator.class_indices.keys()
  print(target_names)
  print(confusion_matrix(generator.classes, row_index))
  print('Classification Report')
  print(classification_report(generator.classes, row_index, target_names=target_names))

In [11]:
test_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data_generator.flow_from_directory("/tmp/smallData/train",
                                                         target_size=(100,100),
                                                         shuffle=False)
test(generator=test_generator, model=model)

Found 95 images belonging to 2 classes.
Instructions for updating:
Please use Model.predict, which supports generators.


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


dict_keys(['bar_chart', 'pie_chart'])
[[39  0]
 [ 0 56]]
Classification Report
              precision    recall  f1-score   support

   bar_chart       1.00      1.00      1.00        39
   pie_chart       1.00      1.00      1.00        56

    accuracy                           1.00        95
   macro avg       1.00      1.00      1.00        95
weighted avg       1.00      1.00      1.00        95



In [12]:
test_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data_generator.flow_from_directory("/tmp/smallData/test",
                                                         target_size=(100,100),
                                                         shuffle=False)

Found 37 images belonging to 2 classes.


In [13]:
test(generator=test_generator, model=model)

dict_keys(['bar_chart', 'pie_chart'])
[[15  4]
 [ 5 13]]
Classification Report
              precision    recall  f1-score   support

   bar_chart       0.75      0.79      0.77        19
   pie_chart       0.76      0.72      0.74        18

    accuracy                           0.76        37
   macro avg       0.76      0.76      0.76        37
weighted avg       0.76      0.76      0.76        37



/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
